<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

## TIPS FOR OPTIMISING SQL QUERIES

## Connecting to our MySQL database

In this train, we'll be using the US Flights database with actual US flights data to illustrate the optimisation techniques with large amounts of data. To follow along, ensure that you have downloaded the database file, `flights.db`. 

In [ ]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


In [ ]:
# Load the Flights database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///flights.db 

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/flights_db_ER.png"  style="width:500px";/>
<br>
<br>
    <em>Figure 1: Flights ERD</em>
</div>

## Use column names instead of `*`

Instead of using  `*` in our select statements, which selects all the columns in the table, we should rather use the specific column names we are interested in.
For example, say we are only interested in the tail number and delay before departure for the first 50 flights. 
 
### Inefficient query

In [ ]:
%%time
%%sql

SELECT * 
FROM flights

### Efficient query

In [ ]:
%%time
%%sql

SELECT TailNum, DepDelay 
FROM flights 
LIMIT 50

### Efficient query

In [ ]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights, carriers
WHERE carriers.Code = flights.UniqueCarrier
LIMIT 2500

### Efficient query

In [ ]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights
INNER JOIN carriers
ON carriers.Code = flights.UniqueCarrier
LIMIT 2500;

### Inefficient query

In [ ]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE '%N7%'
LIMIT 2500;

### Efficient query

In [ ]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE 'N7%'
LIMIT 2500;

### Inefficient query

In [ ]:
%%time
%%sql

SELECT Origin, DepDelay 
FROM flights
WHERE LOWER(Origin) = 'las'
LIMIT 7000;

### Efficient query

In [ ]:
%%time
%%sql

WITH CTE AS
(SELECT LOWER(Origin) Origin_lower, DepDelay FROM flights)

SELECT * FROM CTE WHERE Origin_lower = 'las'
LIMIT 7000;

If we use functions when searching for patterns, the index-based optimisations can’t be used. 
One alternative method is to use CTEs instead.


### Inefficient query

In [ ]:
%%time
%%sql

SELECT TailNum, DepDelay
FROM flights
WHERE AirTime/5 < DepDelay
LIMIT 5000;

### Efficient query

In [ ]:
%%time
%%sql

WITH CTE AS 
(SELECT AirTime/5 as fifth_time, DepDelay, TailNum FROM flights)

SELECT TailNum, DepDelay 
FROM CTE
WHERE CTE.fifth_time < CTE.DepDelay
LIMIT 5000;

Calculated fields in `JOIN` and `WHERE` clauses also prevent indices from being used. CTEs can be used instead as well.